The __tangle__ step in literate programming converts text in the specification language to the source language; the source language must be valid to the object language.  [`pidgin.tangle`](tangle.ipynb) transpiles __Markdown__ to valid `IPython` flavored __Python__ using the `IPython` configuration system.

Literate `pidgin` documents are reusable as __Python__ modules, documentation, and tests.

In [1]:
    import ast, IPython, re, functools, traitlets, doctest, textwrap, mistune, types, importnb, typing as t
    if __name__ == '__main__': 
        %reload_ext pidgin

In [2]:
    class Capture:
        format = None
        def __init__(Capture, code): Capture.format = IPython.get_ipython().display_formatter.format(code)     
        def _repr_mimebundle_(Capture, include=None, exclude=None): return Capture.format

In [3]:
    def define_object(line, code=""):  IPython.get_ipython().user_ns[line] = Capture(code)

In [7]:
    def load_ipython_extension(shell):
        unload_ipython_extension(shell)
        shell.input_transformer_manager.cleanup_transforms.insert(0, tangle.PidginTangle().instance())
        shell.ast_transformers.append(ReturnYieldDisplay())
        shell.register_magic_function(tangle.define_object, 'cell', 'object')
        
        for i, transformer in enumerate(shell.input_transformers_cleanup):
            try:
                if transformer.initial_re.pattern[1:4] == '>>>':
                    shell.input_transformers_cleanup.pop(i)
                    break
            except: ...
                            
        try: IPython.core.magic.register_cell_magic(markdown_to_python)
        except: shell.log.error("Unable to load the pidgin.tangle cell magic.")

In [10]:
    def unload_ipython_extension(shell):
        shell.input_transformer_manager.cleanup_transforms = [
            object for object in shell.input_transformer_manager.cleanup_transforms
            if not isinstance(object, tangle.PidginTangle)]
